# Training and Deploying A Deep Learning Model in Keras MobileNet V2 and Heroku: A Step-by-Step Tutorial 

**This is part of my blog [here](https://medium.com/@malnakli/tf-serving-keras-mobilenetv2-632b8d92983c)**

## Prepare data for training


|Label|Description|  
|-  |      -      |
| 0 | T-shirt/top | 
| 1 | Trouser     | 
| 2 | Pullover    |  
| 3 | Dress       | 
| 4 | Coat        | 
| 5 | Sandal      | 
| 6 | Shirt       |
| 7 | Sneaker     | 
| 8 | Bag         | 
| 9 | Ankle boot  | 


In [ ]:
from keras.datasets.fashion_mnist import load_data

# Load the fashion-mnist train data and test data
(x_train, y_train), (x_test, y_test) = load_data()

print("x_train shape:", x_train.shape, "y_train shape:", y_train.shape)

## Helper function to display imags

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

def show_images(images):
    """
    images : numpy arrays
    """
    n_images = len(images)
    titles = ['(%d)' % i for i in range(1, n_images + 1)]
    num = 5
    iter_num = np.ceil(n_images / num).astype(int)
    for i in range(iter_num):
        fig = plt.figure()
        sub_images = images[i * num:num * (i + 1)]
        sub_titles = titles[i * num:num * (i + 1)]
        for n, (image, title) in enumerate(zip(sub_images, sub_titles)):
            a = fig.add_subplot(1, np.ceil(len(sub_images)), n + 1)
            if image.ndim == 2:
                plt.gray()
            a.set_title(title, fontsize=15)
            plt.imshow(image)


### Show samples of data

In [ ]:
show_images(x_train[:10])

### Data normalization
Normalize the data dimensions so that they are of approximately the same scale.In general, normalization makes very deep NN easier to train, special in Convolutional and Recurrent neural network.

Here is a nice explanation [video](https://www.coursera.org/lecture/deep-neural-network/normalizing-activations-in-a-network-4ptp2) and an [article](https://medium.com/@darrenyaoyao.huang/why-we-need-normalization-in-deep-learning-from-batch-normalization-to-group-normalization-d06ea0e59c17)

In [ ]:
norm_x_train = x_train.astype('float32') / 255
norm_x_test = x_test.astype('float32') / 255
# dsiplay images
show_images(norm_x_train[:10])

### Convert labels (y_train and y_test) to one hot encoding
A one hot encoding is a representation of categorical variables as binary vectors.  
[Here is the full explanation](https://machinelearningmastery.com/how-to-one-hot-encode-sequence-data-in-python/) If you would like to have a deep understanding.

In [ ]:
from keras.utils import to_categorical
encoded_y_train = to_categorical(y_train, num_classes=10, dtype='float32')
encoded_y_test = to_categorical(y_test, num_classes=10, dtype='float32')

### Resize images & convert to 3 channel (RGB)

[MobileNet V2](https://keras.io/applications/#mobilenetv2) model accepts one of the following formats: (96, 96), (128, 128), (160, 160),(192, 192), or (224, 224). In addition, the image has to be 3 channel (RGB) format. Therefore, We need to resize & convert our images. from (28 X 28) to (96 X 96 X 3).

In [ ]:
from skimage.transform import resize

target_size = 96

def preprocess_image(x):
    # Resize the image to have the shape of (96,96)
    x = resize(x, (target_size, target_size),
            mode='constant',
            anti_aliasing=False)
    
    # convert to 3 channel (RGB)
    x = np.stack((x,)*3, axis=-1) 
    
    # Make sure it is a float32, here is why 
    # https://www.quora.com/When-should-I-use-tf-float32-vs-tf-float64-in-TensorFlow
    return x.astype(np.float32)

Running the previous code in all our data data, it may eat up a lot of memory resources; therefore, we are going to use generator.   
[Python Generator](https://www.programiz.com/python-programming/generator) is a function that returns an object (iterator) which we can iterate over (one value at a time).

In [ ]:
from sklearn.utils import shuffle
def load_data_generator(x, y, batch_size=64):
    num_samples = x.shape[0]
    while 1:  # Loop forever so the generator never terminates
        try:
            shuffle(x)
            for i in range(0, num_samples, batch_size):
                x_data = [preprocess_image(im) for im in x[i:i+batch_size]]
                y_data = y[i:i + batch_size]
            
                # convert to numpy array since this what keras required
                yield shuffle(np.array(x_data), np.array(y_data))
        except Exception as err:
            print(err)

## Train a Deep Learning model

In [ ]:
from keras.applications.mobilenetv2 import MobileNetV2
from keras.layers import Dense, Input, Dropout
from keras.models import Model

def build_model( ):
    input_tensor = Input(shape=(target_size, target_size, 3))
    base_model = MobileNetV2(
        include_top=False,
        weights='imagenet',
        input_tensor=input_tensor,
        input_shape=(target_size, target_size, 3),
        pooling='avg')

    for layer in base_model.layers:
        layer.trainable = True  # trainable has to be false in order to freeze the layers
        
    op = Dense(256, activation='relu')(base_model.output)
    op = Dropout(.25)(op)
    
    ##
    # softmax: calculates a probability for every possible class.
    #
    # activation='softmax': return the highest probability;
    # for example, if 'Coat' is the highest probability then the result would be 
    # something like [0,0,0,0,1,0,0,0,0,0] with 1 in index 5 indicate 'Coat' in our case.
    ##
    output_tensor = Dense(10, activation='softmax')(op)

    model = Model(inputs=input_tensor, outputs=output_tensor)


    return model

In [ ]:
from keras.optimizers import Adam
model = build_model()
model.compile(optimizer=Adam(),
              loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])

In [ ]:
train_generator = load_data_generator(norm_x_train, encoded_y_train, batch_size=64)

model.fit_generator(
    generator=train_generator,
    steps_per_epoch=5,
    verbose=1,
    epochs=2)

### Test


In [ ]:
test_generator = load_data_generator(norm_x_test, encoded_y_test, batch_size=64)
model.evaluate_generator(generator=test_generator,
                         steps=900,
                         verbose=1)

## Save the model
Make sure you save the model, because we are going to use in next part

In [ ]:
model_name = "tf_serving_keras_mobilenetv2"
model.save(f"models/{model_name}.h5")

## Part 2 (if you follow the blog)

## Makes Model ready to tensorflow serving

### Tensorflow serving 
[TensorFlow Serving](https://www.tensorflow.org/serving/overview) is a flexible, high-performance serving system for machine learning models, designed for **production** environments.


In [ ]:
from keras.models import load_model
model = load_model(f"models/{model_name}.h5")

### Build & Save model to be tensorflow serving ready


In [ ]:
import os
import tensorflow as tf
import keras
# Import the libraries needed for saving models
# Note that in some other tutorials these are framed as coming from tensorflow_serving_api which is no longer correct
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl

# images will be the input key name
# scores will be the out key name
prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def(
    {
    "images": model.input
    }, {
    "scores": model.output
    })

# export_path is a directory in which the model will be created
export_path = os.path.join(
    tf.compat.as_bytes('models/export/{}'.format(model_name)),
    tf.compat.as_bytes('1'))

builder = saved_model_builder.SavedModelBuilder(export_path)

sess = keras.backend.get_session()

# Add the meta_graph and the variables to the builder
builder.add_meta_graph_and_variables(
    sess, [tag_constants.SERVING],
    signature_def_map={
        'prediction': prediction_signature,
    })
# save the graph
builder.save()

## Setting up Heroku & Docker

- [Install docker for macOS and windows](https://www.docker.com/products/docker-desktop)
- A little more work for Ubuntu users but still straightforward [Install docker for Ubuntu](https://docs.docker.com/install/linux/docker-ce/ubuntu/#upgrade-docker-ce)
- [Signup to Heroku](https://signup.heroku.com/)
- [Install heroku-cli](https://devcenter.heroku.com/articles/heroku-cli#download-and-install)

### After you have installed docker and heroku-cli.

Run the following to make sure docker & heroku have been installed correctly
```
> docker ps 
CONTAINER ID IMAGE COMMAND CREATED STATUS PORTS NAMES

> heroku --version
heroku/7.18.3 darwin-x64 node-v10.12.0

# make sure you have logged in to your heroku account
heroku login
# Output should have:   
Logged in as xxxxx@xxx.xx
```


## Deploy the model to Heroku

#### Download tensorflow serving image from `hub.docker.com`
Because tensorflow serving docker image was not optimized for heroku.   
I have created a dockerfile that following heroku instructure. cleck [here](https://github.com/malnakli/ML/tf_serving_keras_mobilenetv2/Dockerfile) to look at.


Also I have pushed the a docker image that ready to deploy to heroku,
which already have the trained model.

`docker pull malnakli/ml:tf-serving-heroku-1.11`

**To build you own**
Run the following:
make sure you ar in the right folder

`cd ML/tf_serving_keras_mobilenetv2`

Build docker image:
`docker build -t tf-serving-heroku-1.11 .`

Once the image build. you can run it locally if you would like, 
otherwise go to deploy section.     
`docker run -p 8501:8501 -e PORT=8501 -t tf-serving-heroku-1.11`

If you see the following the last output, then it works.    
```
... tensorflow_serving/model_servers/server.cc:301] Exporting HTTP/REST API at:localhost:8501 ...
```
### Deploy

##### Log in to Container Registry:
`heroku container:login`

##### Create a heroku app
`heroku create ${YOUR_APP_NAME}`

#### Push the docker image to heroku
`heroku  container:push web -a ${YOUR_APP_NAME}`   
`heroku container:release web -a ${YOUR_APP_NAME}`

## Call the model

#### The loacl url
> `http://localhost:8501//v1/models/tf_serving_keras_mobilenetv2/versions/1:predict`

#### The Heroku url
> http://https://tf-serving-keras-mobilenetv2.herokuapp.com//v1/models/tf_serving_keras_mobilenetv2/versions/1:predict

JSON data that is sent to TensorFlow Model Server:
```
{
  "signature_name":'prediction',
  "instances": [{"images":image.tolist()}]
}
```

And you can see the full documentation about RESTful API in [Here](https://www.tensorflow.org/serving/api_rest). 

In [ ]:
from urllib import request
from PIL import Image

image_url = "https://cdn.shopify.com/s/files/1/2029/4253/products/Damb_Back_2a3cc4cc-06c2-488e-8918-2e7a1cde3dfc_530x@2x.jpg"
image_path = f"tmp/{image_url.split('/')[-1]}"
# download image
with request.urlopen(url=image_url, timeout=10) as response:
    data = response.read()
    with open(image_path, 'wb') as f:
        f.write(data)

# convert image to grayscale.
image = Image.open(image_path).convert('L')
# resize the image to 28 28 to make sure it is similar to our dataset
image.thumbnail((28,28))
image = preprocess_image(np.array(image))
print(image.shape)
show_images([image])


In [ ]:
import requests
import json
"""
NOTE:
change https://tf-serving-keras-mobilenetv2.herokuapp.com to your url or 
if you ran the docker locally, then replace with http://localhost:8501
""" 
url = "https://tf-serving-keras-mobilenetv2.herokuapp.com"
full_url = f"{url}/v1/models/tf_serving_keras_mobilenetv2/versions/1:predict"
data = {"signature_name":"prediction",
        "instances":[{"images":image.tolist()}]}
data = json.dumps(data)

In [ ]:
import sys
labels = ['T-shirt/top' ,'Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag', 'Ankle boot']
try:
    response = requests.post(full_url,data=data)
    response = response.json()
    highest_index = np.argmax(response['predictions'])
    print(labels[highest_index])
except:
    print(sys.exc_info()[0])

**If you have any suggest or question you can create an issue in this repo**

# References:

Here are some articles that it convert similar aspect cover in this notebook.

- [Fashion-MNIST with tf.Keras](https://medium.com/tensorflow/hello-deep-learning-fashion-mnist-with-keras-50fcff8cd74a)
- [A Comprehensive guide to Fine-tuning Deep Learning Models in Keras  ](https://flyyufelix.github.io/2016/10/03/fine-tuning-in-keras-part1.html)
- [Transfer Learning and Fine Tuning: Let's discuss](https://www.linkedin.com/pulse/transfer-learning-fine-tuning-lets-discuss-arun-das/)
- [Serving Image-Based Deep Learning Models with TensorFlow-Serving’s RESTful API](https://medium.com/@tmlabonte/serving-image-based-deep-learning-models-with-tensorflow-servings-restful-api-d365c16a7dc4)
- [How to Setup Tensorflow Serving For Production](https://medium.com/@brianalois/how-to-setup-tensorflow-serving-for-production-3cc2abf7efa)
- [How to Run Dockerized Apps on Heroku… and it’s pretty sweet](https://medium.com/travis-on-docker/how-to-run-dockerized-apps-on-heroku-and-its-pretty-great-76e07e610e22)